In [1]:
import cv2
import numpy as np
# Load the YOLOv3 model
net = cv2.dnn.readNet("yolov3.weights", "yolov3.cfg")
# Load the CCTV footage
cap = cv2.VideoCapture("footage.mp4")


In [2]:
import cv2
# Specify the path to save the extracted frames
extracted_frame_path = r"C:\Users\HARISH A K\Desktop\python\Deep_Learning\Deep_learning_projects\human_detection_in_footage\frames_path"

# Specify the frame interval to extract (every 12th frame)
frame_interval = 12

frame_count = 0  # Count of all frames processed
extracted_frame_count = 0  # Count of frames actually extracted

while True:
    success, img = cap.read()
    
    if not success:
        print("All frames are processed")
        break
    
    frame_count += 1
    
    if frame_count % frame_interval == 0:
        extracted_frame_count += 1
        filename = extracted_frame_path + "\\frame%d.jpg" % extracted_frame_count
        cv2.imwrite(filename, img)
    

    
# Release the capture
cap.release()

# Print the total number of frames extracted
print("Total frames extracted:", extracted_frame_count)


All frames are processed
Total frames extracted: 71


In [3]:

# Initialize variables for counting people and tracking frames with highest and lowest counts
highest_people_count = 0
highest_frame_number = -1
lowest_people_count = float('inf')
lowest_frame_number = -1



In [4]:
# Loop through each frame
for i in range(1, 72):  # Assuming you have 71 frames extracted
    # Load the extracted frame
    frame_path = extracted_frame_path + "\\frame%d.jpg" % i
    image = cv2.imread(frame_path)
    height, width = image.shape[:2]
    
    # Preprocess the frame for YOLO
    blob = cv2.dnn.blobFromImage(image, 0.00392, (416, 416), swapRB=True, crop=False)
    net.setInput(blob)
    
    # Run YOLO and get the output
    outs = net.forward(net.getUnconnectedOutLayersNames())
    
    # Thresholds for detection
    conf_threshold = 0.5
    nms_threshold = 0.4
    
    # Initialize lists to store detection results
    class_ids = []
    confidences = []
    boxes = []
    
    # Process detections
    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            confidence = scores[class_id]
            
            if confidence > conf_threshold and class_id == 0:  # 0 corresponds to the 'person' class
                center_x = int(detection[0] * width)
                center_y = int(detection[1] * height)
                w = int(detection[2] * width)
                h = int(detection[3] * height)
                x = int(center_x - w / 2)
                y = int(center_y - h / 2)
                
                class_ids.append(class_id)
                confidences.append(float(confidence))
                boxes.append([x, y, w, h])
    
    # Apply non-maximum suppression
    indices = cv2.dnn.NMSBoxes(boxes, confidences, conf_threshold, nms_threshold)
    
    # Get the count of detected people for the current frame
    count = len(indices)
    
    # Update the highest and lowest counts along with their respective frame numbers
    if count > highest_people_count:
        highest_people_count = count
        highest_frame_number = i
    if count < lowest_people_count:
        lowest_people_count = count
        lowest_frame_number = i
# Print frames with highest and lowest people count
if highest_frame_number != -1:
    print(f"Frame {highest_frame_number} has the highest people count: {highest_people_count}")
if lowest_frame_number != -1:
    print(f"Frame {lowest_frame_number} has the lowest people count: {lowest_people_count}")


Frame 3 has the highest people count: 5
Frame 14 has the lowest people count: 2


In [10]:
import cv2

# Loop through each frame
for i in range(1, 72):
    # Load the extracted frame
    frame_path = extracted_frame_path + "\\frame%d.jpg" % i
    image = cv2.imread(frame_path)
    # Draw bounding boxes around detected people
    for index in indices:
        #index = index[0]
        x, y, w, h = boxes[index]
        color = (0, 255, 0)  # BGR color for the bounding box (green)
        cv2.rectangle(image, (x, y), (x + w, y + h), color, 2)
    
    # Display the frame with bounding boxes
    cv2.imshow('Video with Bounding Boxes', image)
    
    # Wait for a short duration and handle key presses
    key = cv2.waitKey(280)
    
    # Press 'q' to exit playback
    if key == ord('q'):
        break

# Close OpenCV windows
cv2.destroyAllWindows()
